In [94]:
from vote_system import confront_dataset, filter_pairs, round, get_website_ids
from parsing import parse_stable, parse_gemma, parse_qwen_coder

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
list_tuple = confront_dataset(["<meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">", "salut"], 
                 ["<meta name=\"viewport\" content=\"width=device-width>", "salut moi"])

In [96]:
list_tuple

[('<meta name="viewport" content="width=device-width, initial-scale=1.0">',
  '<meta name="viewport" content="width=device-width>',
  0.7777777777777778),
 ('<meta name="viewport" content="width=device-width, initial-scale=1.0">',
  'salut moi',
  0.0),
 ('salut', '<meta name="viewport" content="width=device-width>', 0.0),
 ('salut', 'salut moi', 0.6666666666666666)]

In [97]:
filter_pairs(list_tuple)

[('<meta name="viewport" content="width=device-width, initial-scale=1.0">',
  '<meta name="viewport" content="width=device-width>',
  0.7777777777777778)]

In [98]:
codes = parse_stable("datasets/dataset_stable-code_intermediate.json", "1c_ru")
for code in codes : 
    print(code)


In [99]:
code_qwen = parse_qwen_coder("datasets/dataset_qwen2-5-coder_intermediate.json", "1c_ru")

In [92]:
code_qwen

['@media (max-width: 768px) {',
 'html, body {     margin: 0;     padding: 0; }',
 'body {     font-family: Arial, sans-serif; }',
 '.container {     width: 90%;     max-width: 1200px;     margin: 0 auto; }',
 'nav ul {     list-style-type: none;     padding: 0;     display: flex;     justify-content: space-between; }',
 '@media (max-width: 576px) {     nav ul {         flex-direction: column;     } }',
 'section, footer {     padding: 20px; }',
 'footer {     background-color: #333;     color: white;     text-align: center;     position: fixed;     width: 100%;     bottom: 0; }']

In [100]:
code_gemma = parse_gemma("datasets/dataset_gemma_intermediate.json", "1c_ru")

In [101]:
code_gemma

['<',
 '**HTML:**',
 '',
 '- Use media queries with breakpoints for different devices.',
 '- Utilize flexible layouts with flexible boxes or grid systems.',
 '- Include responsive image sizes with the `srcset',
 '',
 '',
 '**CSS:**',
 '',
 '- Use relative units like `%` or `em',
 '- Employ responsive navigation menus that collapse on smaller screens.',
 '- Design for different viewport sizes and adjust layouts accordingly.',
 '- Use responsive font sizes that adapt to screen sizes.',
 '- Consider using responsive plugins like Bootstrap or Foundation.',
 '']

In [102]:
ids = get_website_ids("datasets/dataset_stable-code_intermediate.json")

In [87]:
for i in range(10):
    print(ids[i])
    cl_sc = parse_stable("datasets/dataset_stable-code_intermediate.json", ids[i])
    cl_gemma = parse_gemma("datasets/dataset_gemma_intermediate.json", ids[i])

    print(filter_pairs(confront_dataset(cl_sc, cl_gemma), threshold=0.1))

    print('=====================')

166_com
[('<meta name="viewport" content="width=device-width, initial-scale=1.0">', '@media (max-width: 600px)', 0.13333333333333333), ('<meta name="viewport" content="width=device-width, initial-scale=1.0">', '<meta name="viewport" content="width=device-width, initial-scale=1.0">', 1.0), ('<meta name="viewport" content="width=device-width, initial-scale=1.0">', 'width: 100%', 0.15384615384615385), ('width: 100%;', '@media (max-width: 600px)', 0.3333333333333333), ('width: 100%;', '<meta name="viewport" content="width=device-width, initial-scale=1.0">', 0.15384615384615385), ('width: 100%;', 'width: 100%', 1.0), ('initial-scale', '<meta name="viewport" content="width=device-width, initial-scale=1.0">', 0.3076923076923077), ('width', '@media (max-width: 600px)', 0.4), ('width', '<meta name="viewport" content="width=device-width, initial-scale=1.0">', 0.16666666666666669), ('width', 'width: 100%', 0.6666666666666666), ('device-width', '@media (max-width: 600px)', 0.3333333333333333), ('d

In [103]:
def tournament_for_website(website_id, threshold=0.1):
    cl_sc = parse_stable("datasets/dataset_stable-code_intermediate.json", website_id)
    cl_gemma = parse_gemma("datasets/dataset_gemma_intermediate.json", website_id)
    
    round1_winner = round(cl_sc, cl_gemma, threshold=threshold)
    
    return round1_winner

In [106]:
tournament_for_website("166_com", threshold=0.1)

['<meta name="viewport" content="width=device-width, initial-scale=1.0">',
 'width: 100%;',
 'initial-scale',
 'width',
 'device-width']